# Section 0

QPM: Assignment 6

DENG Yunyun <br>
LI Jiaxin <br>
SBAI Ilyas <br>
ZHOU Zhichen <br>

Note: NA

# Section 0.bis

We retrieve the price data for the 6 stocks from January 2015 until December 2022 using the library yfinance. We transform the daily data into monthly data and store it in the dataframe ```prices```

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

# Date range
start_date = "2015-01-01"
end_date = "2022-12-31"

# Tickers' name
tic = "AAPL MSFT AMZN NVDA TSLA META"

# Importing the stock data using the download function from the yf package
prices = (yf.download(tickers=tic,
start=start_date,
end=end_date)
.reset_index()
.rename(columns = {"Date": "date",
                   "Open": "open",
"High": "high",
"Low": "low",
"Close": "close",
"Adj Close": "adjusted",
"Volume": "volume"
})
)

# Keeping only the price data
prices = prices.set_index('date').adjusted

# Monthly returns
log_returns = np.log(prices).diff()
ret = log_returns.resample('M').sum()
ret.head()

[*********************100%***********************]  6 of 6 completed


,AAPL,AMZN,META,MSFT,NVDA,TSLA
date,,,,,,
2015-01-31,0.069169,0.139006,-0.032913,-0.146198,-0.047301,-0.074329
2015-02-28,0.096016,0.069799,0.039520,0.089036,0.142699,-0.001278
2015-03-31,-0.031874,-0.021430,0.040331,-0.075530,-0.052582,-0.074350
2015-04-30,0.005770,0.125321,-0.042866,0.179201,0.058909,0.180227
2015-05-31,0.044341,0.017509,0.005318,-0.030804,0.001460,0.103900


Defining the market weights following the Investopedia article and computing the excess returns, which are simply the sample mean of the monthly returns.

In [2]:
# Defining the market weights, following the order of our prices dataframe
indx_wgt = np.array((0.071,0.0324,0.0184,0.0651,0.0284,0.0187))

#Rescale the weights so that they sum to one
indx_wgt = indx_wgt/(np.ones(6).T @ indx_wgt)

# Excess returns are the mean of the returns
exc_ret = ret.mean().to_numpy()

# Displaying the excess returns, in percents
for i, ex_ret in enumerate(exc_ret):
    print(f"Asset {i + 1}: {ex_ret*100:.2f}%")

Asset 1: 1.73%
Asset 2: 1.77%
Asset 3: 0.45%
Asset 4: 1.84%
Asset 5: 3.55%
Asset 6: 2.22%


# Section 1

We first compute the market gamma that is defined as: <br>
$$ \gamma_{mkt}=\frac{SR}{\sigma_{mkt}} $$

We first compute the returns of the market portfolio by multiplying the returns of each stock by its weight, for each date.
We then compute the Sharpe ratio of this series of returns and store it in ```SR_mkt```. Note that since the returns have been annualised in the first step, we do not annualise the Sharpe ratio again.

In [3]:
# Returns of the market portfolio
mkt_ret = np.array(ret) @ indx_wgt

# Sharpe ratio of the market portfolio
SR_mkt = np.mean(mkt_ret) / np.std(mkt_ret)

# Market gamma
gamma_mkt = SR_mkt / np.std(mkt_ret)
print('The market gamma is:',gamma_mkt)

The market gamma is: 3.808140595776307


We then compute the Markowitz portfolio weights using this market gamma: <br><br>
$$ w_{Markowitz}={{1}\over\gamma_{mkt}}\Sigma^{-1}\mu_{sample} $$ 

In [4]:
# Covariance matrix of the excess returns
sigma =ret.cov().to_numpy()

# Invert the covariance matrix
sigma_inv = np.linalg.inv(sigma)

# Compute the Markowitz weights
wgts_Markowitz = (1 / gamma_mkt) * (sigma_inv @ exc_ret)

# Displaying the results in percents
for i, weight in enumerate(wgts_Markowitz):
    print(f"Asset {i + 1}: {weight*100:.3f}")

Asset 1: 8.601
Asset 2: 3.919
Asset 3: -33.806
Asset 4: 95.336
Asset 5: 29.090
Asset 6: 0.171


# Section 2
We compute the CAPM-implied expected returns:
$$ \mu_{CAPM}=\gamma_{mkt}\Sigma w_{mkt} $$

In [5]:
# Computing the CAPM-implied expected returns
mu_capm = gamma_mkt * (sigma @ indx_wgt)

# Displaying the results, in percents
for i, exp_ret in enumerate(mu_capm):
    print(f"Asset {i + 1}: {exp_ret*100:.2f}%")

Asset 1: 1.95%
Asset 2: 1.90%
Asset 3: 1.49%
Asset 4: 1.44%
Asset 5: 2.80%
Asset 6: 2.94%


# Section 3

We define the P matrix, the q vector and the omega matrix.
Each line in the P matrix corresponds to a single view. Each column correspond to the assets, in the same order as our prices dataframe. A 1 indicate a positive view and a -1 a negative view. When a row contains both a +1 and a -1, it indicates a relative outperformance of the asset indicated by the +1. <br>
The q vector contains the numbers corresponding to our views: the values of the absolute and relative outperformance.<br>
The omega matrix is defined as:
$$ \Omega=P_{k}(\tau\Sigma)P_{k}^{T} $$
where k = {1,2,3,4} is the row of the matrix P.

In [6]:
# P matrix that contains our views
P = np.matrix([[1,0,0,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,-1],[0,0,1,0,0,-1]])

# q vector that contains the values corresponding to our views
q = np.array([0.1,0.05,0.02,0.01])

# omega matrix which captures the uncertainty in the views
tau_sigma = (1/len(ret)) * sigma
omega = np.zeros((4,4))
for i in range(4):
    omega[i][i] = P[i] @ tau_sigma @ P[i].T
omega

array([[7.19796748e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.24165854e-05, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 3.16522544e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.94913613e-04]])

# Section 4

We compute the posterior expected returns ```mu_bl```:
$$ \mu_{BL}=[(\tau\Sigma)^{-1}+P^{T}\Omega^{-1} P]^{-1}[(\tau\Sigma)^{-1}\mu_{CAPM}+P^{T}\Omega^{-1}q] $$

In [7]:
# Computing the inverse of the product (tau)*sigma
tau_sigma_inv = np.linalg.inv(tau_sigma)

# Computing the inverse of the uncertainty matrix omega
omga_inv = np.linalg.inv(omega)

# Computing mu_bl by separating the first and second facto
factor1 = np.linalg.inv(tau_sigma_inv + P.T @ omga_inv @ P)
factor2 = np.reshape(tau_sigma_inv @ mu_capm,(1,-1)) + P.T @ omga_inv @ q
mu_bl = factor1 * factor2.T
print('The conditional expected excess return is:')

# Displaying the posterior expected excess returns, in percents
for i, retu in enumerate(np.asarray(mu_bl).reshape(-1)):
    print(f"Asset {i + 1}: {retu*100:.2f}%")


The conditional expected excess return is:
Asset 1: 6.18%
Asset 2: 4.81%
Asset 3: 4.35%
Asset 4: 4.02%
Asset 5: 7.45%
Asset 6: 5.78%


We compute the posterior covariance matrix of returns ```sigma_bl```:
$$ \Sigma_{BL}=\Sigma+[(\tau\Sigma)^{-1}+P^{T}\Omega^{-1}P]^{-1} $$

In [8]:
sigma_bl = sigma + np.linalg.inv(tau_sigma_inv + P.T @ omga_inv @ P)
sigma_bl

matrix([[0.00694197, 0.0041054 , 0.00313391, 0.00313395, 0.0064204 ,
         0.00717561],
        [0.0041054 , 0.00822341, 0.00401553, 0.00355848, 0.00664713,
         0.00649313],
        [0.00313391, 0.00401553, 0.0096323 , 0.00282456, 0.00445212,
         0.00439478],
        [0.00313395, 0.00355848, 0.00282456, 0.00409108, 0.00516703,
         0.00449033],
        [0.0064204 , 0.00664713, 0.00445212, 0.00516703, 0.01761465,
         0.00734237],
        [0.00717561, 0.00649313, 0.00439478, 0.00449033, 0.00734237,
         0.02759244]])

# Section 5

We use the previously computed ```mu_bl``` and ```sigma_bl``` to compute the mean-variance weights.

$$ w_{BL}=\gamma_{mkt}\Sigma_{BL}^{-1}\mu_{BL} $$ 

In [9]:
# Computing the mean variance weights using the previously defined mu_bl and sigma_bl
wgt_BL = (1/gamma_mkt) * (np.linalg.inv(sigma_bl) @ mu_bl)

# Displaying the mean variance weights, in percents
for i, wgt in enumerate(np.asarray(wgt_BL).reshape(-1)):
    print(f"Asset {i + 1}: {wgt*100:.2f}%")

Asset 1: 174.83%
Asset 2: 13.70%
Asset 3: 30.11%
Asset 4: 91.29%
Asset 5: 14.93%
Asset 6: -17.33%


The weights from Black-Litterman model incorporates subjective views of investors along with equilibrium returns. <br> <br> Consequently, these weights tend to be higher on assets anticipated to yield great returns compared to their counterparts in both the CAPM and sample moments weights, such as AAPL, MSFT and NVDA. <br><br> Similarly, these weights tend to be lower on assets anticipated to yield low returns compared to their counterparts in both the CAPM and sample moments weights, such as TSLA. 